# Search Retrieval System
## CSC 575 Final Project
### Ayesha Ali, Swathi Babu

In [11]:
import os
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import string
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from ast import literal_eval
import json
import tarfile
import matplotlib.pyplot as plt
from ast import literal_eval
import tkinter as tk
from tkinter import scrolledtext
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
inverted_index1={}
docLengths1={}
def crawl_directory(filepath):
    Ndoc=0
    for directory, subdirlist, filelist in os.walk(filepath):
        dirList = []
        for f in filelist:
            #print('\t' + f)
            #print('sub',subdirlist)
            if f=='.DS_Store':
                continue
            else:
                try:
                    
                    docid = f

                    doc = preprocess_words(open(directory+'/'+f).read())
                    #print('print doc',directory+'/'+f)
                    Ndoc+=1


                    create_inverted_index(doc, docid,inverted_index1)
                except UnicodeDecodeError:
                    continue
    doc_lengths(inverted_index1, docLengths1, Ndoc)
    with open('inverted_index1.json', 'w') as f:
        json.dump(inverted_index1, f)
    with open('doc_lengths1.json', 'w') as f:
        json.dump(docLengths1, f)
    print(len(inverted_index1))
    return Ndoc

In [12]:
def preprocess_words(line):
    '''Function to preprocess the tokens'''
    line = line.translate(str.maketrans('', '', string.punctuation)) #Removing Punctuation
    line = line.lower().strip() #making it lower case and removing additional spaces
    stopWords = set(stopwords.words('english')) #stopwords
    tokens = word_tokenize(line) #creating a list of words
    words = [x for x in tokens if not x in stopWords] #removing stopwords
    #Stemming
    ps = PorterStemmer()
    for i in range(len(words)):
        words[i] = ps.stem(words[i])
    #returning the list of words/tokens
    return(words)

In [13]:
def tf_idf(dict, N):
    ''' to find the idf values of the given term'''
    idf = np.log2(N / len(dict))  
    return idf

In [14]:
def create_inverted_index(doc, docid, inverted_index):
    ''' to create the inverted index for given set of documents'''
    for token in doc:
        #if token not in dict, add the token
        if token not in inverted_index:
            inverted_index[token] = {}
        #for each token, a nested dict containing doc id and count in that doc
        if docid not in inverted_index[token]:
            inverted_index[token][docid] = 0
        #incrementing count by 1 for that doc id
        inverted_index[token][docid] += 1

In [15]:
def doc_lengths(inverted_index, docLengths, NDoc):
    ''' to create dictionary storing the document lengths for the given set of documents'''
    #Incrementing the Document lengths
    for i in inverted_index:
        idf = tf_idf(inverted_index[i], NDoc)
        for j in inverted_index[i]:
            if j in docLengths.keys():
                docLengths[j] += ((inverted_index[i][j] * idf)**2)
            else:
                docLengths[j] = ((inverted_index[i][j] * idf)**2)
        for w in docLengths:
            docLengths[w] = np.sqrt(docLengths[w])

In [16]:
def readfile(filepath, linestart, inverted_index, docLengths):
    '''Reads through the set of documents and calls the function that creates inverted index and doc length dictionaries'''
    with open(filepath) as file:
        line = file.readline()
        docid = None
        doc = [] #tokens of document #
        NDoc = 0 #number of documents
        while line:
            if line.startswith(linestart):
                NDoc+=1
                create_inverted_index(doc, docid, inverted_index) #sending the previous document tokens and the id to update the inverted index
                doc = [] #reloading the doc list to store tokens for the next document
                try:
                    docid = int(line.strip().replace(linestart, '' )) #processing on document id to get user friendly form
                except ValueError:
                    docid = line.strip().replace(linestart, '' )
                line = file.readline() 
            else:
                doc +=(preprocess_words(line))
            line = file.readline()
    create_inverted_index(doc, docid, inverted_index)
    doc_lengths(inverted_index, docLengths, NDoc)
    return(NDoc)

In [17]:
def query_process(query): 
    '''Pre-processes the query and creates the query vector in dictionary form'''
    p_query = preprocess_words(query)
    #CALCULATING THE QUERY DICTIONARY 
    QtermDict = {}
    for k in p_query:
        if k in QtermDict.keys():
            QtermDict[k]+=1
        else:
            QtermDict[k] = 1
    return p_query, QtermDict

In [18]:
#CALCULATING THE DOT PRODUCT SCORES FOR ONE QUERY
def retrieval_scores(p_query, QtermDict, NDoc, inverted_index):
    '''Calculates the retrieval scores for the documents for a given query'''
    rankDict = {} #Scores for the documents
    Qlength = 0 #Calculating query lengths
    for i in set(p_query): #going through the query terms
        try:
            idf = tf_idf(inverted_index[i], NDoc) #Calculating idf of the term
            Qlength+= (idf**2) #incrementing the query length
            for j in (inverted_index[i]): #going through each document in the postings and incrementing the respective dictionaries
            #Incrementing the document scores
                if j in rankDict.keys():
                    rankDict[j] += (inverted_index[i][j] * idf)*(QtermDict[i] * idf)
                else:
                    rankDict[j] = (inverted_index[i][j] * idf)*(QtermDict[i] * idf)
        except:
              continue
    #Applying square root for Query length
    Qlength = np.sqrt(Qlength)
    return (Qlength, rankDict)

In [19]:
#RANKING BASED ON COSINE FOR THE GIVEN QUERY
def cosine_Ranking(rankDict, Qlength,docLengths):
    '''Calculates the cosine ranking and sorts based on it for a given query'''
    cosineRank = {}
    for r in rankDict:
        cosineRank[r] = rankDict[r]/(Qlength*docLengths[r])
    #Sorting the dictionary based on ranking
    cosineRank_sorted = dict(sorted(cosineRank.items(), key = lambda item: item[1], reverse = True))
    return cosineRank_sorted

In [20]:
def modify_query_rocchio(Q, R, NR, alpha, beta):
    '''Runs standard IR Rocchio relevance feedback method'''
    mean_R = np.mean(R, axis=0)
    mean_NR = np.mean(NR, axis=0)
    #rochio formula
    Q1= Q+(alpha*mean_R)-(beta*mean_NR)
    Q1 = np.maximum(Q1, 0) #converting negative entries to 0
    return Q1

In [1]:
def rocchio_process(rankedDocs, rel, QtermDict,k,inverted_index):
    '''Creates the TD matrix with just the top 10 retrieved documents to use for rocchio method and creates a query vector after rocchio'''
    retDocs = list(rankedDocs.keys())[:k] #Top 10 retrieved documents
    #creating a dataframe to create vectors of the relevant and non-relevant docs
    DT = pd.DataFrame(index = retDocs) 
    for i in inverted_index:
        for j in inverted_index[i]:
            if j in retDocs:
                DT.loc[j,i] = inverted_index[i][j]
    DT.fillna(0, inplace = True)
    terms = DT.columns #terms in these docs
    Qdict = QtermDict.copy() #creating the new query doc (used for retrieval)
    #If the terms in query don't exist in the documents
    for j in Qdict:
        if j not in terms:
            DT.loc[:,j] = 0
    #creating the new query doc (used for retrieval)
    for x in terms:
        if x not in Qdict.keys():
            Qdict[x] = 0
    #Query vector
    Q = np.array(list(Qdict.values()))
    #Relevant documents matrix
    #relRet = set(retDocs) & set(rel)#relevant documents retrieved
    relRet = list(set(list(DT.index)) & set(rel))
    R = np.array([DT.loc[i,:] for i in relRet])
    #Non-Relevant documents matrix
    notrel = list(set(list(DT.index)) - set(rel))
    NR = np.array([DT.loc[i,:] for i in notrel])
    #Calling the Rocchio function
    alpha = 0.5
    beta = 0.25
    Q1 = modify_query_rocchio(Q, R, NR, alpha, beta)
    #Creating a new query dictionary with the Rocchio results
    for i in range(len(terms)):
        Qdict[terms[i]] = Q1[i]
    return (terms,Qdict)

In [2]:
def feedback_user(QtermDict, relevant,k,rankedDocs,inverted_index,docLengths,NDoc):
    '''Runs the rocchio process and then using the new query vector, finds the new retrieved documents'''
    #Running the rocchio process
    terms, Qdict =rocchio_process(rankedDocs, relevant, QtermDict, k,inverted_index)
    #getting the new query vector and getting scores for the new query
    Qlength1, rankDict1 = retrieval_scores(terms,Qdict, NDoc, inverted_index)
    newrankedDocs = cosine_Ranking(rankDict1, Qlength1,docLengths)
    return (newrankedDocs)

In [3]:
def print_title(i_num): #Function to print the title of the given doc id --> specific to MED files
    '''Function to print the title of MED documents'''
    text = ""
    with open('/Users/admin/Desktop/courses2/IR_Project/med/MED.ALL', "r") as f:
        line = f.readline().strip()
        while line:
            if line.startswith(".I"):
                current_i_num = line.split()[1]
                if current_i_num == i_num:
                  # Found the desired I number, skip lines until ".W"
                    line = f.readline().strip()
                    while not line.startswith(".W"):
                        line = f.readline().strip()

                  # Read lines until first full stop 
                  #1st line
                    line = f.readline().strip()
                    if line.endswith('.'):
                        text=line
                        break    
                  #if the title is more than one line
                    text=''
                    while line and not line.startswith("."):
                        text += line + " "
                        line = f.readline().strip()
                        if "." in line:
                            text += line[:line.index(".")] + " "
                            break
                  # Finished reading text for desired I number, exit loop
                    break
            line = f.readline().strip()
    return(text)

In [4]:
def printDocs(rankedDocs):
    '''Function to print the top 10 documents'''
    n = 10 #Number of documents to retrieve
    for i in rankedDocs:
        if n>0:
            print('\nDocument', i)
            print_title(i)
            print('\n')
            n-=1

In [5]:
def first_time(filepath = '/Users/admin/Desktop/courses2/IR_Project/med/MED.ALL'):
    '''Function to run for the first time'''
    inverted_index = {}
    doc_lengths = {}
    NDoc = readfile(filepath, '.I', inverted_index, doc_lengths)
    # Save dictionary to file using JSON
    with open('inverted_index.json', 'w') as f:
        json.dump(inverted_index, f)
    with open('doc_lengths.json', 'w') as f:
        json.dump(doc_lengths, f)

In [6]:
def precision_recall(ret, rel):
    '''Calculates the precision and recall'''
    #ret = [int(a) for a in ret]
    com = set(ret) & set(rel)
    recall = len(com)/len(rel)
    precision = len(com)/ len(ret)
    return precision, recall

In [7]:
def query_eval_10():
    '''Presicion and Recall for the 30 test queries'''
    pres = []
    rec = []
    pres_roc = []
    rec_roc = []
    D = pd.read_csv('/Users/admin/Desktop/courses2/IR_Project/med/MED.REL', header = None, sep = ' ', index_col = 0)
    with open('inverted_index.json', 'r') as f:
        inverted_index = json.load(f)
    with open('doc_lengths.json', 'r') as f:
        docLengths = json.load(f)
    NDoc = len(docLengths)
    f = open('/Users/admin/Desktop/courses2/IR_Project/med/MED.QRY')
    line = f.readline()
    docid = int(line[3:].strip())
    line = f.readline()
    line = f.readline()
    query=''
    while line:
        if line.startswith('.I'):
            #The previous query is completely read so running the system for that query
            print(docid)
            print(query)
            p_query, QtermDict = query_process(query)
            Qlength, rankDict = retrieval_scores(p_query, QtermDict, NDoc,inverted_index)
            rankedDocs = cosine_Ranking(rankDict, Qlength,docLengths)
            ret = list(rankedDocs.keys())[0:10]
            rel = list(D.loc[docid,2])
            rel = [str(x) for x in rel]
            i,j = precision_recall(ret,rel)
            pres.append(i)
            rec.append(j)
          #After Rocchio 
            newrankedDocs = feedback_user(QtermDict, rel, 10,rankedDocs,inverted_index,docLengths,NDoc)
            ret_new = list(newrankedDocs.keys())[0:10]
            i1,j1 = precision_recall(ret_new,rel)
            pres_roc.append(i1)
            rec_roc.append(j1)
            #Initializing for the next query
            docid = int(line[3:].strip())
            query = ''
            line = f.readline()
        else:
            query = query + line
        line = f.readline()
    print(docid)
    print(query)
    p_query, QtermDict = query_process(query)
    Qlength, rankDict = retrieval_scores(p_query, QtermDict, NDoc, inverted_index)
    rankedDocs = cosine_Ranking(rankDict, Qlength,docLengths)
    ret = list(rankedDocs.keys())[0:10]
    rel = list(D.loc[docid,2])
    i,j = precision_recall(ret,rel)
    pres.append(i)
    rec.append(j)
    newrankedDocs = feedback_user(QtermDict, rel,10,rankedDocs,inverted_index,docLengths,NDoc)
    ret_new = list(newrankedDocs.keys())[0:10]
    i1,j1 = precision_recall(ret_new,rel)
    pres_roc.append(i1)
    rec_roc.append(j1)
    print('Before Rocchio:')
    print('The Precision value:', sum(pres)/len(pres))
    print('The Recall value:',sum(rec)/len(rec))
    print('\nAfter Rocchio:')
    print('The Precision value:', sum(pres_roc)/len(pres_roc))
    print('The Recall value:', sum(rec_roc)/len(rec_roc))
    f.close()

In [8]:
def precision_Recall_plot():
    '''Plot for finding ideal number of retrieved documents'''
    with open('inverted_index.json', 'r') as f:
        inverted_index = json.load(f)
    with open('doc_lengths.json', 'r') as f:
        docLengths = json.load(f)
    NDoc = len(docLengths)
    pres_k = []
    rec_k =[]
    pres_k_roc =[]
    rec_k_roc = []
    for k in range(5,16): #k is the number of retrieved documents
        pres = []
        rec = []
        pres_roc = []
        rec_roc = []
        f = open('/Users/admin/Desktop/courses2/IR_Project/med/MED.QRY')
        D = pd.read_csv('/Users/admin/Desktop/courses2/IR_Project/med/MED.REL', header = None, sep = ' ', index_col = 0)
        line = f.readline()
        docid = int(line[3:].strip())
        line = f.readline()
        line = f.readline()
        query=''
        while line:
            if line.startswith('.I'):
                #The previous query is completely read so running the system for that query
                p_query, QtermDict = query_process(query)
                Qlength, rankDict = retrieval_scores(p_query, QtermDict, NDoc,inverted_index)
                rankedDocs = cosine_Ranking(rankDict, Qlength,docLengths)
                ret = list(rankedDocs.keys())[0:k]
                rel = list(D.loc[docid,2])
                rel = [str(x) for x in rel]
                i,j = precision_recall(ret,rel)
                pres.append(i)
                rec.append(j)
                #After Rocchio 
                newrankedDocs = feedback_user(QtermDict, rel,k,rankedDocs,inverted_index,docLengths,NDoc)
                ret_new = list(newrankedDocs.keys())[0:k]
                i1,j1 = precision_recall(ret_new,rel)
                pres_roc.append(i1)
                rec_roc.append(j1)
                #Initializing for the next query
                docid = int(line[3:].strip())
                query = ''
                line = f.readline()
            else:
                query = query + line
            line = f.readline()
        p_query, QtermDict = query_process(query)
        Qlength, rankDict = retrieval_scores(p_query, QtermDict, NDoc, inverted_index)
        rankedDocs = cosine_Ranking(rankDict, Qlength,docLengths)
        ret = list(rankedDocs.keys())[0:k]
        rel = list(D.loc[docid,2])
        i,j = precision_recall(ret,rel)
        pres.append(i)
        rec.append(j)
        
        newrankedDocs = feedback_user(QtermDict, rel,k,rankedDocs,inverted_index,docLengths,NDoc)
        ret_new = list(newrankedDocs.keys())[0:k]
        i1,j1 = precision_recall(ret_new,rel)
        pres_roc.append(i1)
        rec_roc.append(j1)
        print(k)
        pres_k.append(sum(pres)/len(pres))
        rec_k.append(sum(rec)/len(rec))
        pres_k_roc.append(sum(pres_roc)/len(pres_roc))
        rec_k_roc.append( sum(rec_roc)/len(rec_roc))
        #print(sum(pres)/len(pres), sum(rec)/len(rec))
        #print(sum(pres_roc)/len(pres_roc), sum(rec_roc)/len(rec_roc))
        f.close()
        
    #Before Rocchio pres rec plot
    plt.plot(range(5, 16), pres_k, label='Precision')
    plt.plot(range(5, 16), rec_k, label='Recall')
    plt.xlabel('Number of Documents')
    plt.ylabel('Values')
    plt.title('Precision vs Recall - BEFORE ROCCHIO')
    plt.legend()
    plt.show()
    #After Rocchio pres rec plot
    plt.plot(range(5, 16), pres_k_roc, label='Precision')
    plt.plot(range(5, 16), rec_k_roc, label='Recall')
    plt.xlabel('Number of Documents')
    plt.ylabel('Values')
    plt.title('Precision vs Recall - AFTER ROCCHIO')
    plt.legend()
    plt.show()
    
    return pres_k,rec_k,pres_k_roc, rec_k_roc

In [9]:
def main():
    '''Main function of the system'''
    #LOAD INVERTED INDEX AND DOCUMENT LENGTHS
    try:
        with open('inverted_index.json', 'r') as f:
            inverted_index = json.load(f)
        with open('doc_lengths.json', 'r') as f:
            docLengths = json.load(f)
    except:
        first_time()
        #crawl_directory('/Users/admin/Desktop/courses2/IR_Project/20_newsgroups')
    NDoc = len(docLengths)
    #INITIALIZE GUI WINDOW
    window = tk.Tk()
    window.title("CSC 575 Information Retrieval System")
    window.geometry("1000x1600")

    #ADD LABELS AND TEXT BOXES TO GUI WINDOW
    query_label = tk.Label(window, text="Enter your Query:")
    query_label.pack()
    query_entry = tk.Entry(window,width=60)
    query_entry.pack()

    output_label = tk.Label(window, text="Search Results:")
    output_label.pack()
    output_text = scrolledtext.ScrolledText(window, height=20)
    output_text.pack()

    #HANDLE SEARCH BUTTON CLICK
    def search():
        query = query_entry.get()
        p_query, QtermDict = query_process(query)
        Qlength, rankDict = retrieval_scores(p_query, QtermDict, NDoc, inverted_index)
        rankedDocs_2 = cosine_Ranking(rankDict, Qlength,docLengths)
        #output_text.delete("1.0", tk.END)
        n=10
        for i in rankedDocs_2:
            if n>0:
                output_text.insert(tk.END, f"\nDocument {i}\n")
                output_text.insert(tk.END, print_title(i))
                output_text.insert(tk.END, '\n'+'-'*70)
                n-=1
        
        feedback_label = tk.Label(window, text="Would you like to provide feedback on the relevance of the documents? (Y/N)")
        feedback_label.pack()
        feedback_entry = tk.Entry(window)
        feedback_entry.pack()
        
        

        def feedback():
            
            def feedback1():
               
                relevant = literal_eval(relevant_entry.get())
                relevant = [str(x) for x in relevant]
                if relevant:
                    newrankedDocs = feedback_user(QtermDict, relevant, 10,rankedDocs_2,inverted_index,docLengths,NDoc)
                    #output_text.delete("1.0", tk.END)
                    n=10
                    output_label1 = tk.Label(window, text="Search Results:")
                    output_label1.pack()
                    output_text1 = scrolledtext.ScrolledText(window, height=20)
                    output_text1.pack()

                    for i in newrankedDocs:
                        if n>0:
                            output_text1.insert(tk.END, f"\nDocument {i}\n")
                            output_text1.insert(tk.END, print_title(i))
                            output_text1.insert(tk.END, '\n'+'-'*70)
                            n-=1
            
            if  feedback_entry.get()=='Y':
                relevant_label = tk.Label(window, text="Please type out the IDs of the relevant documents from above [docid1, docid2,..]:")
                relevant_label.pack()
                relevant_entry = tk.Entry(window,width=60)
                relevant_entry.pack()
                
                feedback_button1 = tk.Button(window, text="Submit Relevant docs", command=feedback1)
                feedback_button1.pack()
            
        feedback_button = tk.Button(window, text="Submit Feedback", command=feedback)
        feedback_button.pack()
    #TO CALL MAIN FUNCTION AND CONTINUE THE LOOP
    def callMain():
        window.destroy()
        main()
        
    #ADD SEARCH BUTTON TO GUI WINDOW
    search_button = tk.Button(window, text="Search", command=search)
    search_button.pack()

    #ADD QUIT BUTTON TO GUI WINDOW
    quit_button = tk.Button(window, text="Click for new Search", command= callMain)
    quit_button.pack()

    #START GUI EVENT LOOP
    window.mainloop()


In [21]:
main()